In [50]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io as sio
import glob
from scipy.interpolate import interp1d
import sys
sys.path.append("./")

In [51]:
data_path = "Hafting2008/C43035A4-5CC5-44F2-B207-126922523FD9_1/"

data_path_list = glob.glob(data_path+"*.mat")

#The naming convention for the files is as follow
#Rat number - session number - file type (i.e: 11015-13120410_EEG)

mice_ids = np.unique([dp.split("/")[-1][:5] for dp in data_path_list])
session_per_animal = {}
cell_id_per_session = {}
for m_id in mice_ids:
    m_paths_list = glob.glob(data_path + m_id + "*.mat")
    sessions = np.unique([dp.split("/")[-1].split("-")[1][:8]  for dp in m_paths_list]).astype(str)
    session_per_animal[m_id] = {}
    for sess in sessions:
        s_paths_list = glob.glob(data_path + m_id + "-" + sess + "*.mat")
        cell_ids = np.unique([dp.split("/")[-1].split(".")[-2][-4:] for dp in s_paths_list]).astype(str)
        session_per_animal[m_id][sess] = {}
        for cell_id in cell_ids:
            r_path = s_paths_list = glob.glob(data_path + m_id + "-" + sess + "*" + cell_id + "*.mat")
            session_per_animal[m_id][sess][cell_id] = sio.loadmat(r_path[0])
            
#Here we record for a specifique mice
recording_paths = sorted(glob.glob(data_path+mice_ids[0]+"-*.mat"))



In [52]:
mouse_data = {}
for r_path in recording_paths:
    trial_id = r_path.split(".")[0][-4:]
    print(trial_id)
    mouse_data[trial_id] = sio.loadmat(r_path)

_EEG
_POS
t5c1


In [53]:
def clean_data(data, keep_headers=False):
    # print(list(data.keys()))
    aux_dict = {}
    for key, val in data.items():
        if isinstance(val, bytes) or isinstance(val, str) or key=="__globals__":
            if keep_headers:
                aux_dict[key] = val
            continue
        else:
            print(len(val))
            if np.isnan(val).any():
                aux_dict[key] = val
            else:
                # Interpolate nans
                x_range = np.linspace(0, 1, num=len(val))
                nan_indexes = np.logical_not(np.isnan(val))[:, 0]
                clean_x = x_range[nan_indexes]
                clean_val = np.array(val)[nan_indexes, 0]
                print(clean_x.shape, clean_val.shape)
                f = interp1d(clean_x, clean_val, kind='cubic')
                aux_dict[key] = f(x_range)
    return aux_dict